# Introduction

In this notebook, I study the identification of MACD divergences from Min|Max points in ZIGZAG indicator.

In [1]:
import sys

####################################################################################
# Data handling 
import pandas as pd
from pandas import concat
from pandas.plotting import scatter_matrix
import numpy as np

####################################################################################
# Visualization
import matplotlib.pyplot as plt
from matplotlib import dates, ticker
from matplotlib.dates import (MONDAY, DateFormatter, MonthLocator, WeekdayLocator, date2num)
import matplotlib as mpl
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import plotly.tools as tls
plotly.offline.init_notebook_mode(connected=True)

####################################################################################
# TA-Lib: 
import talib

####################################################################################
# Other utilities
import datetime
import time
import os
import sys
import math
from enum import Enum

print('Packages loaded!!')

Packages loaded!!


In [2]:
# Append path for MACD_Signal_Listener class (parent folder)
sys.path.append('..')

# import main class and event class
from MACD_Signal_Listener import MACD_Signal_Listener, MACD_Events
from ZIGZAG_Signal_Listener import ZIGZAG_Signal_Listener

In [3]:
# loads df dataframe
df = pd.read_csv('../csv_data/EURUSD_M15.csv', sep=';')
df = df[['OPEN','HIGH','LOW','CLOSE']]

In [4]:
# creates a MACD Signal Listener with default parameters
msl = MACD_Signal_Listener(param_num_bars_per_swing=5, param_num_bars_per_minmax_wdow=2)

# creates zigzag signal listener
zz = ZIGZAG_Signal_Listener()

In [14]:
def datafeed_update(n):
  last_sample = -2000
  execution_count = n

  # setup plotting figure with 2 rows and 1 column
  fig = plotly.tools.make_subplots(rows=2, cols=1, subplot_titles=('Price', 'MACD'), shared_xaxes=True, vertical_spacing=0.1)
  fig['layout'].update(height=600, title='Price & MACD', xaxis=dict(rangeslider=dict(visible = False)))

  # updates datafeed samples
  _from_sample = last_sample + execution_count
  _to_sample = _from_sample + 1000

  # executes MACD signal generator
  _events, _df_result = msl.MACD( df[_from_sample : _to_sample],
                                  applied = 'CLOSE', 
                                  fastperiod=15, 
                                  slowperiod=26, 
                                  signalperiod=9,
                                  common_event_listeners=[],
                                  bullish_divergence_listeners=[],
                                  bearish_divergence_listeners=[],
                                  bullish_main_zero_cross_listeners=[],
                                  bearish_main_zero_cross_listeners=[],
                                  bullish_main_signal_cross_listeners=[],
                                  bearish_main_signal_cross_listeners=[])


  #executes ZIGZAG indicator
  _df_result, _x, _y, _events = zz.ZIGZAG(_df_result, 
                                    minbars=12, 
                                    bb_period=20, 
                                    bb_dev = 2.0, 
                                    nan_value = 0.0, 
                                    verbose=False)
  
  # Plot results
  _df_draw = _df_result
  curr_sample = 0
  last_sample = _df_draw.shape[0] - 1
  
  # build candlestick trace
  trace_ohlc = go.Ohlc(x=_df_draw.index.values, open=_df_draw.OPEN, high=_df_draw.HIGH, low=_df_draw.LOW, close=_df_draw.CLOSE, name='Candlestick')
  fig.append_trace(trace_ohlc, 1, 1)
  
  _dfz = _df_result[_df_result.ZIGZAG > 0].copy()
  trace_zigzag = go.Scatter(x=_dfz.reset_index()['index'], y=_dfz.ZIGZAG, name='zigzag', line=scatter.Line(color='black', width=1))
  fig.append_trace(trace_zigzag, 1, 1)

  # build MACD traces
  trace_macd = go.Scatter(x=_df_draw.index.values, y=_df_draw.MACD, name='MACD', line=scatter.Line(color='blue', width=1))
  fig.append_trace(trace_macd, 2, 1)
  trace_macdsig = go.Scatter(x=_df_draw.index.values, y=_df_draw.MACDS, name='MACD_sig', line=scatter.Line(color='red', width=1))
  fig.append_trace(trace_macdsig, 2, 1)

  # build BEAR_DIVERGENCES on both charts (black)
  # select divergence zones
  div_traces_row1 = []
  div_traces_row2 = []
  # get divergences as an array and just select those in the plotting range
  bear_div_zones = msl.getBearishDivergences()
  if len(bear_div_zones) > 0:
    _bear_div_zones = np.asarray(bear_div_zones)
    _bear_div_zones = _bear_div_zones[(_bear_div_zones[:,0] >= curr_sample) & (_bear_div_zones[:,0] <= last_sample)]
    #print('Bearish Divergence zones={}'.format(len(_bear_div_zones)))
    # for each one, build a pair of traces, one for each row
    for d in _bear_div_zones:
      #print('added bear_div at {}'.format(d - curr_sample))
      # add trace to row1 list
      div_traces_row1.append(go.Scatter(
              x= d - curr_sample, 
              y= [_df_draw.HIGH.at[d[0]], _df_draw.HIGH.at[d[1]]],     
              name='bear_div_r1 at {}'.format(d - curr_sample),
              line=scatter.Line(color='black', width=1)))
      # add trace to row2 list
      div_traces_row2.append(go.Scatter(
              x= d - curr_sample, 
              y= [_df_draw.MACD.at[d[0]], _df_draw.MACD.at[d[1]]],     
              name='bear_div_r2 at {}'.format(d - curr_sample),
              line=scatter.Line(color='black', width=1)))


  # build BULL_DIVERGENCES on both charts (orange)
  # get divergences as an array and just select those in the plotting range
  bull_div_zones = msl.getBullishDivergences()
  if len(bull_div_zones) > 0:
    _bull_div_zones = np.asarray(bull_div_zones)
    _bull_div_zones = _bull_div_zones[(_bull_div_zones[:,0] >= curr_sample) & (_bull_div_zones[:,0] <= last_sample)]
    #print('Bullish Divergence zones={}'.format(len(_bull_div_zones)))
    # for each one, build a pair of traces, one for each row
    for d in _bull_div_zones:
      #print('added bull_div at {}'.format(d - curr_sample))
      # add trace to row1 list
      div_traces_row1.append(go.Scatter(
              x= d - curr_sample, 
              y= [_df_draw.LOW.at[d[0]], _df_draw.LOW.at[d[1]]],     
              name='bull_div_r1 at {}'.format(d - curr_sample),
              line=scatter.Line(color='orange', width=1)))
      # add trace to row2 list
      div_traces_row2.append(go.Scatter(
              x= d - curr_sample, 
              y= [_df_draw.MACD.at[d[0]], _df_draw.MACD.at[d[1]]],     
              name='bull_div_r2 at {}'.format(d - curr_sample),
              line=scatter.Line(color='orange', width=1)))

  for d in div_traces_row1:
    fig.append_trace(d, 1, 1)
  for d in div_traces_row2:
    fig.append_trace(d, 2, 1)
  return fig



In [15]:
step = 0

In [16]:
fig = datafeed_update(step)
step += 1
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

